In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import pickle
import logging
from itertools import combinations
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
lgcg_path = os.path.abspath(os.path.join('../../../lazified-generalized-conditional-gradient/lgcg'))
if lgcg_path not in sys.path:
    sys.path.append(lgcg_path)
from sisso import SISSO
from lgcg import LGCG_finite

# Load Data

In [2]:
K = pickle.load(open("evals_thermal.pkl", "rb"))
K.shape

(233799, 75)

In [3]:
y = pd.read_csv("thermal_conductivity_data.csv")["log kappa_L"].values
y.shape

(75,)

# SISSO

In [4]:
exp_sisso = SISSO(K=K.T, target=y)

In [5]:
exp_sisso.fit(max_iterations=4)

INFO:root:Iteration: 1
INFO:root:Error: 0.3037392401377368
INFO:root:Number of combinations: 25
INFO:root:Optimal combination: (18075,)
INFO:root:Optimal coefficients: [11.06288295  1.02545477]
INFO:root:Time of iteration: 0.046790122985839844
INFO:root:------------------
INFO:root:Iteration: 2
INFO:root:Error: 0.23759122318118397
INFO:root:Number of combinations: 1225
INFO:root:Optimal combination: (17654, 55857)
INFO:root:Optimal coefficients: [ 9.27143569 -2.28259027  3.81452901]
INFO:root:Time of iteration: 0.2444324493408203
INFO:root:------------------
INFO:root:Iteration: 3
INFO:root:Error: 0.17777442750366648
INFO:root:Number of combinations: 67525
INFO:root:Optimal combination: (17654, 55857, 228993)
INFO:root:Optimal coefficients: [ 9.9177373  -2.45898648 -1.60420282  4.08647698]
INFO:root:Time of iteration: 4.1302995681762695
INFO:root:------------------
INFO:root:Iteration: 4
INFO:root:Error: 0.1638370934699184
INFO:root:Number of combinations: 3921225
INFO:root:Optimal com

[array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.11840932]),
 array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.44046387]),
 array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.47186572]),
 array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.57010248])]

# LGCG

In [6]:
exp = LGCG_finite(K=K.T, target=y, alpha=0.1)

In [8]:
result = exp.solve(tol=1e-11)

DEBUG:root:SSN in 2 dimensions converged in 2 iterations to tolerance 1.000E-01
INFO:root:1: Phi 3.644E-01, epsilon 2.550E+00, support [ 46695 233799], Psi 1.000E-01
DEBUG:root:SSN in 3 dimensions converged in 1 iterations to tolerance 1.000E-01
INFO:root:2: Phi 1.864E-01, epsilon 3.195E+00, support [ 19855  46695 233799], Psi 1.000E-01
DEBUG:root:SSN in 4 dimensions converged in 1 iterations to tolerance 1.000E-01


INFO:root:3: Phi 1.565E-01, epsilon 3.250E+00, support [ 19855  46695 153142 233799], Psi 1.000E-01
DEBUG:root:SSN in 5 dimensions converged in 1 iterations to tolerance 1.000E-01
INFO:root:4: Phi 1.210E-01, epsilon 3.107E+00, support [ 19855  46695  85007 153142 233799], Psi 1.000E-01
DEBUG:root:SSN in 6 dimensions converged in 1 iterations to tolerance 1.000E-01
INFO:root:5: Phi 5.902E-02, epsilon 2.905E+00, support [ 19855  85007 114978 153142 233799], Psi 1.000E-01
DEBUG:root:SSN in 6 dimensions converged in 0 iterations to tolerance 1.000E-01
INFO:root:6: Phi 5.193E-02, epsilon 2.695E+00, support [ 19855  85007 110190 114978 153142 233799], Psi 1.000E-01
DEBUG:root:SSN in 6 dimensions converged in 0 iterations to tolerance 5.000E-02
INFO:root:7: Phi 4.574E-02, epsilon 2.497E+00, support [ 19855  85007 110190 114978 153142 233799], Psi 5.000E-02
DEBUG:root:SSN in 6 dimensions converged in 1 iterations to tolerance 2.500E-02
INFO:root:8: Phi 5.933E-02, epsilon 2.316E+00, support [ 1

In [9]:
u = result["u"]
print(u)
support = result["support"]
print(support)
u_bar = np.zeros(exp.K.shape[1])
for ind, val in zip(support, u):
    u_bar[ind] = val

[ 5.63073546e-01  4.30162553e-04  5.31586378e-03  8.12262120e-03
 -8.50070612e-02  5.29192754e-03 -4.64990300e-01  1.75735707e-04
  1.14962932e-04  1.46636015e-03  2.82628591e-03  6.31656823e-03]
[  3664  18331  19073  21404  36431  37457  46171  74388  75210 110190
 126955 149760]


In [ ]:
# Renormalize u_bar
# u_bar = u_bar/exp.target_norm
# for iter, nor in enumerate(exp.K_norms):
#     u_bar[iter] *= nor

In [10]:
# RMSError wrt target
np.sqrt(np.mean(np.square(np.matmul(np.append(K.T, np.ones((K.shape[1], 1)), axis=1), u_bar) - y)))

0.24840872239600692

### Correlations

In [13]:
K_support = exp.K[:, support]
K_support.shape

(75, 11)

In [14]:
df =pd.DataFrame(K_support)

In [15]:
corr = df.corr()
corr.style.background_gradient(cmap='BrBG_r', axis=None).format(precision=2)

DEBUG:matplotlib:matplotlib data path: /vol/cs-hu/hnatiuar@hu-berlin.de/miniconda3/envs/sissopp_env/lib/python3.9/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/vol/cs-hu/hnatiuar@hu-berlin.de/.config/matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:CACHEDIR=/vol/cs-hu/hnatiuar@hu-berlin.de/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /vol/cs-hu/hnatiuar@hu-berlin.de/.cache/matplotlib/fontlist-v330.json


,0,1,2,3,4,5,6,7,8,9,10
0,1.00,0.58,0.59,0.39,-0.77,-0.92,0.63,0.54,0.47,0.56,0.61
1,0.58,1.00,0.88,0.69,-0.60,-0.74,0.96,0.98,0.65,0.82,0.84
2,0.59,0.88,1.00,0.60,-0.51,-0.70,0.86,0.86,0.85,0.77,0.89
3,0.39,0.69,0.60,1.00,-0.26,-0.57,0.69,0.68,0.41,0.42,0.59
4,-0.77,-0.60,-0.51,-0.26,1.00,0.72,-0.64,-0.59,-0.43,-0.78,-0.50
5,-0.92,-0.74,-0.70,-0.57,0.72,1.00,-0.76,-0.70,-0.49,-0.63,-0.74
6,0.63,0.96,0.86,0.69,-0.64,-0.76,1.00,0.97,0.64,0.80,0.77
7,0.54,0.98,0.86,0.68,-0.59,-0.70,0.97,1.00,0.66,0.79,0.77
8,0.47,0.65,0.85,0.41,-0.43,-0.49,0.64,0.66,1.00,0.67,0.78
9,0.56,0.82,0.77,0.42,-0.78,-0.63,0.80,0.79,0.67,1.00,0.77


### SISSO Step

In [16]:
# SO
error = 10
for n in range(1,len(support)+1):
    min_error = 10 * error
    combinatorial_combinations = combinations(support, n)
    optimal_combination = None
    optimal_coefficients = None
    combinatorial_counter = 0
    for combination in combinatorial_combinations:
        combinatorial_counter += 1
        if exp.K.shape[1]-1 in support: # Constant term is already in the support
            submatrix = exp.K[:, np.array(combination)]
        else:
            submatrix = np.append(exp.K[:, np.array(combination)], np.ones((exp.K.shape[0], 1)), axis=1)
        try:
            least_squares, res, rank, s = np.linalg.lstsq(
                submatrix, y, rcond=None
            )
            local_error = np.sqrt(
                np.mean(
                    np.square(np.matmul(submatrix, least_squares) - y)
                )
            )  # RMSE
        except np.linalg.LinAlgError:
            local_error = min_error
        if local_error < min_error:
            min_error = local_error
            optimal_combination = combination
            optimal_coefficients = least_squares
    error = min_error

    logging.info(f"Iteration: {n}")
    logging.info(f"Error: {error}")
    logging.info(f"Number of combinations: {combinatorial_counter}")
    logging.info(f"Optimal combination: {optimal_combination}")
    logging.info(f"Optimal coefficients: {optimal_coefficients}")
    logging.info("------------------")

INFO:root:Iteration: 1
INFO:root:Error: 0.34652639094464455
INFO:root:Number of combinations: 11
INFO:root:Optimal combination: (149760,)
INFO:root:Optimal coefficients: [13.24568038 -0.18019397]
INFO:root:------------------
INFO:root:Iteration: 2
INFO:root:Error: 0.24529862495500807
INFO:root:Number of combinations: 55
INFO:root:Optimal combination: (19073, 46171)
INFO:root:Optimal coefficients: [ 6.38343945 -5.226458    0.0979673 ]
INFO:root:------------------
INFO:root:Iteration: 3
INFO:root:Error: 0.20590683397571385
INFO:root:Number of combinations: 165
INFO:root:Optimal combination: (3664, 75210, 149760)
INFO:root:Optimal coefficients: [ 3.92391854  3.49652666  6.13769723 -0.12596177]
INFO:root:------------------
INFO:root:Iteration: 4
INFO:root:Error: 0.19873944223662543
INFO:root:Number of combinations: 330
INFO:root:Optimal combination: (3664, 75210, 110190, 149760)
INFO:root:Optimal coefficients: [ 3.96084385  3.30661799  1.55174112  4.95399608 -0.14549949]
INFO:root:--------

INFO:root:------------------
INFO:root:Iteration: 7
INFO:root:Error: 0.1824317242001786
INFO:root:Number of combinations: 330
INFO:root:Optimal combination: (3664, 18331, 19073, 21404, 36431, 75210, 149760)
INFO:root:Optimal coefficients: [ 2.62309049 -4.30774824  1.83661171  1.13710739 -1.65992088  5.04153592
  6.40590392 -0.20380974]
INFO:root:------------------
INFO:root:Iteration: 8
INFO:root:Error: 0.18194086682365085
INFO:root:Number of combinations: 165
INFO:root:Optimal combination: (3664, 18331, 19073, 21404, 36431, 75210, 110190, 149760)
INFO:root:Optimal coefficients: [ 2.70419694 -3.4472118   1.38594985  1.16144992 -1.5749378   4.44188589
  0.60232498  5.95842242 -0.19915728]
INFO:root:------------------
INFO:root:Iteration: 9
INFO:root:Error: 0.1813068119643968
INFO:root:Number of combinations: 55
INFO:root:Optimal combination: (3664, 18331, 19073, 21404, 36431, 46171, 75210, 110190, 149760)
INFO:root:Optimal coefficients: [ 1.98854963 -3.12614072  1.19670116  1.0761242  -

# Exact GCG

In [82]:
exp_exact = LGCG_finite(K=K.T, target=y, alpha=0.04)

In [83]:
result_exact = exp_exact.solve_exact(tol=1e-11)

In [84]:
u = result_exact["u"]
#print(u)
support = result_exact["support"]
print(support)
u_bar = np.zeros(exp_exact.K.shape[1])
for ind, val in zip(support, u):
    u_bar[ind] = val

[ 10437  15021  19073  21404  30460  30969  36431  46171  57949  64164
  80063 104733 110177 115192 126955 149757 149760 149856 153055 178344
 182230 204717 204758]


In [85]:
# Renormalize u_bar
u_bar_r = u_bar/exp_exact.target_norm
for iter, nor in enumerate(exp_exact.K_norms):
    u_bar_r[iter] *= nor
u_bar_r[support]

array([ 0.01751896,  0.00071934,  0.0718112 ,  0.04073621, -0.02381583,
       -0.03765706, -0.05225023, -0.21733367, -0.03015952,  0.01318575,
        0.04737146, -0.08157477,  0.11908302,  0.02529143,  0.00835627,
        0.01920027,  0.1926848 ,  0.01243408,  0.00051576,  0.01295019,
        0.01996613,  0.00680695,  0.00938611])

In [86]:
p_bar = np.abs(exp_exact.p(u_bar_r))

In [87]:
for x in support:
    print(p_bar[x])

0.03999999999999999
0.039999999999999925
0.03999999999999761
0.04000000000000017
0.04000000000000052
0.039999999999999675
0.04000000000000012
0.03999999999999994
0.04000000000000001
0.0400000000000009
0.040000000000000355
0.0399999999999999
0.040000000000000466
0.040000000000000854
0.03999999999999952
0.03999999999999968
0.04000000000000099
0.040000000000000195
0.04000000000000001
0.040000000000000736
0.03999999999999965
0.04000000000000087
0.03999999999999862


In [88]:
# Sigma in our theoretocal considerations of the finite setting
0.5*(0.04-np.max(p_bar[p_bar<0.03999]))

5.842530288544673e-06

In [89]:
# Error wrt target
np.sqrt(np.mean(np.square(np.matmul(np.append(K.T, np.ones((K.shape[1], 1)), axis=1), u_bar) - y)))

0.16110076734079748

### Correlations

In [90]:
K_support = exp_exact.K[:, support]
K_support.shape

(75, 23)

In [91]:
df =pd.DataFrame(K_support)

In [92]:
corr = df.corr()
corr.style.background_gradient(cmap='BrBG_r', axis=None).format(precision=2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1.00,0.98,0.12,0.35,-0.07,-0.17,0.06,-0.11,0.11,0.25,0.18,0.04,-0.14,0.02,0.16,-0.09,0.29,0.43,0.08,0.24,0.44,0.31,0.18
1,0.98,1.00,0.14,0.40,-0.09,-0.17,0.05,-0.15,0.10,0.28,0.21,0.00,-0.16,0.01,0.16,-0.12,0.31,0.43,0.07,0.30,0.49,0.36,0.23
2,0.12,0.14,1.00,0.60,-0.66,-0.66,-0.51,-0.70,0.01,0.91,0.86,-0.59,0.69,0.81,0.77,0.24,0.89,0.74,0.57,0.48,0.68,0.64,0.46
3,0.35,0.40,0.60,1.00,-0.62,-0.51,-0.26,-0.57,0.03,0.60,0.61,-0.36,0.20,0.29,0.42,-0.24,0.59,0.71,0.29,0.71,0.91,0.92,0.81
4,-0.07,-0.09,-0.66,-0.62,1.00,0.85,0.57,0.59,0.03,-0.55,-0.69,0.35,-0.36,-0.56,-0.70,0.04,-0.53,-0.46,-0.54,-0.58,-0.59,-0.73,-0.66
5,-0.17,-0.17,-0.66,-0.51,0.85,1.00,0.66,0.60,-0.12,-0.54,-0.68,0.34,-0.47,-0.73,-0.78,-0.23,-0.55,-0.58,-0.74,-0.43,-0.58,-0.57,-0.40
6,0.06,0.05,-0.51,-0.26,0.57,0.66,1.00,0.72,0.11,-0.41,-0.51,0.66,-0.37,-0.68,-0.78,-0.27,-0.50,-0.39,-0.77,-0.37,-0.34,-0.34,-0.33
7,-0.11,-0.15,-0.70,-0.57,0.59,0.60,0.72,1.00,-0.01,-0.72,-0.77,0.87,-0.27,-0.54,-0.63,0.05,-0.74,-0.69,-0.50,-0.66,-0.66,-0.66,-0.65
8,0.11,0.10,0.01,0.03,0.03,-0.12,0.11,-0.01,1.00,0.00,0.17,0.21,-0.03,-0.04,-0.04,-0.15,-0.02,0.08,-0.00,0.02,0.06,0.11,0.03
9,0.25,0.28,0.91,0.60,-0.55,-0.54,-0.41,-0.72,0.00,1.00,0.76,-0.62,0.51,0.60,0.63,0.07,0.82,0.64,0.43,0.58,0.75,0.66,0.51


# Cross Validation

In [75]:
def get_sisso_errors(X, y, X_test, y_test):
    exp = SISSO(X, y)
    n_solutions = exp.fit(max_iterations=3)
    X_test = np.append(X_test, np.ones((X_test.shape[0], 1)), axis=1)
    errors = [np.sqrt(np.mean(np.square(np.matmul(X_test,solution)-y_test))) for solution in n_solutions]  #RMSE
    del exp
    return errors

def get_gcg_errors(X,y,X_test,y_test, alpha=0.1, combinatorial_depth=3):
    exp = LGCG_finite(K=X, target=y, alpha=alpha)
    result = exp.solve_exact(tol=1e-11)
    u_bar = result["u"]
    support = result["support"]
    ones_norm = np.linalg.norm(np.ones((X_test.shape[0],1)))
    X_test = np.append(X_test, np.ones((X_test.shape[0], 1)), axis=1)
    X = np.append(X, np.ones((X.shape[0], 1)), axis=1)
    errors = [np.sqrt(np.mean(np.square(np.matmul(X_test[:,support],u_bar)-y_test)))]
    optimal_combinations = []

    error = 1e14
    if exp.K.shape[1]-1 not in support: # Constant is not in the support
        support = np.append(support, exp.K.shape[1]-1)
    for n in range(1,min(combinatorial_depth+1, len(support)+1)):
        min_error = 10 * error
        combinatorial_combinations = combinations(support, n)
        optimal_combination = None
        optimal_coefficients = None
        for combination in combinatorial_combinations:
            submatrix = X[:, np.array(combination)]
            try:
                least_squares, res, rank, s = np.linalg.lstsq(
                    submatrix, y, rcond=None
                )
                local_error = np.sqrt(
                    np.mean(
                        np.square(np.matmul(submatrix, least_squares) - y)
                    )
                )  # RMSE
            except np.linalg.LinAlgError:
                local_error = min_error
            if local_error < min_error:
                min_error = local_error
                optimal_combination = np.array(combination)
                optimal_coefficients = least_squares
        error = min_error
        errors.append(np.sqrt(np.mean(np.square(np.matmul(X_test[:,optimal_combination], optimal_coefficients) - y_test))))
        optimal_combinations.append(optimal_combination)

    del exp
    return errors, optimal_combinations

In [76]:
X = K.T
print(X.shape)

(75, 233799)


In [77]:
logging.getLogger().setLevel(logging.CRITICAL) # Supress logging

In [78]:
columns = ["sisso_1", "sisso_2", "sisso_3", "gcg_all", "gcg_1", "gcg_2", "gcg_3", "gcg_4", "gcg_5", "gcg_6", "gcg_7", "gcg_8"]
cv = KFold(n_splits=10, random_state=7, shuffle=True)
errors = []
for i, (train_index, test_index) in enumerate(cv.split(X)):
    print(f"Split {i}, train size {len(train_index)}, test size {len(test_index)}")
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    sisso_errors = get_sisso_errors(X_train, y_train, X_test, y_test)
    gcg_errors, optimal_combinations = get_gcg_errors(X_train, y_train, X_test, y_test, alpha=0.04, combinatorial_depth=8)
    combined_errors = sisso_errors+gcg_errors
    # print(combined_errors)
    errors.append(combined_errors)
error_df = pd.DataFrame(errors, columns=columns)

Split 0, train size 67, test size 8
Split 1, train size 67, test size 8
Split 2, train size 67, test size 8
Split 3, train size 67, test size 8
Split 4, train size 67, test size 8
Split 5, train size 68, test size 7
Split 6, train size 68, test size 7
Split 7, train size 68, test size 7
Split 8, train size 68, test size 7
Split 9, train size 68, test size 7


In [79]:
error_df

,sisso_1,sisso_2,sisso_3,gcg_all,gcg_1,gcg_2,gcg_3,gcg_4,gcg_5,gcg_6,gcg_7,gcg_8
0,0.447166,0.326363,0.210053,0.279777,0.462850,0.390815,0.287809,0.245553,0.260917,0.223082,0.265496,0.211832
1,0.281492,0.307067,0.214096,0.157027,0.157616,0.303559,0.250761,0.172619,0.200102,0.183457,0.181472,0.156261
2,0.307585,0.198688,0.205380,0.242017,0.435424,0.161704,0.428045,0.359262,0.317791,0.138951,0.252728,0.246163
3,0.255499,0.363984,0.223242,0.376679,0.362957,0.327303,0.556813,0.531726,0.453874,0.358704,0.336915,0.309787
4,0.431701,0.412297,0.258570,0.219320,1.121610,0.382852,0.387924,0.477284,0.558348,0.165823,0.272678,0.251724
5,0.202724,0.264904,0.126728,0.223090,0.549367,0.426031,0.379616,0.160952,0.142260,0.156944,0.133739,0.131128
6,0.362688,0.281513,0.234432,0.253742,0.391240,0.253092,0.247024,0.246382,0.171122,0.185815,0.196842,0.194502
7,0.257493,0.190936,0.199664,0.180706,0.361454,0.219486,0.211035,0.198335,0.219168,0.196368,0.188547,0.173564
8,0.276093,0.225918,0.165432,0.241749,0.311730,0.384640,0.320224,0.265719,0.227669,0.217152,0.251249,0.267232
9,0.331224,0.245872,0.264488,0.212296,0.389717,0.273037,0.254609,0.246014,0.236361,0.230826,0.220661,0.223318


In [80]:
error_df.mean()

sisso_1    0.315366
sisso_2    0.281754
sisso_3    0.210209
gcg_all    0.238640
gcg_1      0.454397
gcg_2      0.312252
gcg_3      0.332386
gcg_4      0.290385
gcg_5      0.278761
gcg_6      0.205712
gcg_7      0.230033
gcg_8      0.216551
dtype: float64

In [94]:
error_df.std()

sisso_1    0.078717
sisso_2    0.071702
sisso_3    0.041049
gcg_all    0.059928
gcg_1      0.255654
gcg_2      0.085604
gcg_3      0.106298
gcg_4      0.126289
gcg_5      0.131251
gcg_6      0.061350
gcg_7      0.057741
gcg_8      0.054477
dtype: float64

In [81]:
optimal_combinations

[array([149760]),
 array([ 46171, 110190]),
 array([ 30969,  46171, 149760]),
 array([ 30969,  46171,  57949, 149760]),
 array([ 30969,  46171,  57949, 110177, 149760]),
 array([ 30969,  46171,  57949, 110177, 149760, 178344]),
 array([ 30969,  46171,  57949,  80063, 110177, 149760, 178344]),
 array([ 30969,  46171,  57949, 110190, 149757, 149760, 178344, 233799])]